**<div style="text-align: right;">Uqaily, Raafay</div>**


**FINM 35000 - Topics in Economics**
<br>
Autumn 2024

<h2><center> Data Project 2: Portfolio Selection and Transformations </center></h2>

### Import Statements

In [47]:
import os
import wrds
import quandl
import zipfile
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import adfuller

### Section 3.1

S&P500 monthly returns data from wrds

In [5]:
db = wrds.Connection()

query = """
    SELECT date, sprtrn AS sp500_ret
    FROM crsp.msi
    WHERE date BETWEEN '1976-01-01' AND '2023-12-31'
"""
sp500_data = db.raw_sql(query)

sp500_data['date'] = pd.to_datetime(sp500_data['date'])
sp500_data.set_index('date', inplace=True)

sp500_data

Enter your WRDS username [raafayuqaily]: raafayu
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


,sp500_ret
date,
1976-01-30,0.118306
1976-02-27,-0.011402
1976-03-31,0.030689
1976-04-30,-0.010995
1976-05-28,-0.014364
...,...
2023-08-31,-0.017716
2023-09-29,-0.048719
2023-10-31,-0.021980


In [44]:
portfolio_returns = pd.read_csv("portfolio_returns.csv")

portfolio_returns['Date'] = pd.to_datetime(portfolio_returns['Date'])
portfolio_returns = portfolio_returns[portfolio_returns['Date'] >= '1976-01-01']
portfolio_returns.reset_index(drop=True, inplace=True)

portfolio_returns['Portfolio_Return'] = portfolio_returns.iloc[:, 1:].mean(axis=1)
portfolio_returns

,Date,ARCHER-DANIELS-MIDLAND CO,BANK OF AMERICA CORP,CONAGRA BRANDS INC,COLGATE-PALMOLIVE CO,CAMPBELL SOUP CO,JOHNSON & JOHNSON,JPMORGAN CHASE & CO,KELLANOVA,COCA-COLA CO,LILLY (ELI) & CO,MERCK & CO,PFIZER INC,PROCTER & GAMBLE CO,TYSON FOODS INC -CL A,WALMART INC,Portfolio_Return
0,1976-01-01,-0.0291,0.1154,0.2500,-0.0238,0.0549,0.0710,0.1818,0.0988,0.1277,0.1087,0.0469,0.0769,0.0393,0.0442,0.0381,0.080053
1,1976-02-01,-0.0380,0.0690,0.1647,0.0269,-0.0250,-0.1100,-0.0210,-0.0857,-0.0889,-0.0928,-0.0690,-0.0966,-0.0774,0.0508,0.1101,-0.018860
2,1976-03-01,-0.0625,0.0649,-0.0204,-0.0349,-0.0037,0.0528,0.0991,-0.0526,0.0478,0.0556,0.1089,0.1194,0.0663,0.0161,0.0840,0.036053
3,1976-04-01,0.0278,-0.0510,-0.0313,-0.0610,-0.0599,-0.0279,-0.0464,0.0741,-0.0612,-0.0709,-0.0319,-0.0879,-0.0133,0.0317,-0.1221,-0.035413
4,1976-05-01,0.0832,-0.1501,0.0761,-0.0680,0.0040,-0.0072,0.0347,-0.0057,-0.0152,-0.0236,-0.0295,0.0229,-0.0042,0.0000,-0.0435,-0.008407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2023-08-01,-0.0613,-0.0966,-0.0893,-0.0366,-0.0899,-0.0278,-0.0736,-0.0788,-0.0339,0.2217,0.0218,-0.0189,-0.0125,-0.0354,0.0208,-0.026020
572,2023-09-01,-0.0489,-0.0450,-0.0823,-0.0321,-0.0149,-0.0367,-0.0090,-0.0247,-0.0567,-0.0308,-0.0486,-0.0625,-0.0549,-0.0522,-0.0165,-0.041053
573,2023-10-01,-0.0510,-0.0380,-0.0022,0.0631,-0.0073,-0.0476,-0.0339,-0.0939,0.0091,0.0313,-0.0024,-0.0787,0.0350,-0.0820,0.0218,-0.018447
574,2023-11-01,0.0365,0.1667,0.0468,0.0486,-0.0057,0.0506,0.1224,0.0521,0.0427,0.0690,-0.0021,0.0105,0.0233,0.0211,-0.0472,0.042353


This portfolio consists of 15 stocks from diverse sectors, providing a balanced exposure to various industries. It includes well-established companies like Archer-Daniels-Midland Co, Bank of America Corp, Johnson & Johnson, JPMorgan Chase & Co, and Coca-Cola Co, which are leaders in agriculture, financials, healthcare, and consumer staples. Additionally, it features global consumer brands such as Procter & Gamble Co and Colgate-Palmolive Co, known for their stability and resilience during economic downturns. The portfolio also incorporates healthcare giants like Pfizer Inc, Merck & Co, and Eli Lilly & Co, which add a defensive element with consistent growth potential. Retail is represented by Walmart Inc and Campbell Soup Co, offering reliable performance in both bull and bear markets. The inclusion of Conagra Brands Inc and Tyson Foods Inc further diversifies the portfolio with exposure to the food industry. This portfolio assumes an equally weighted allocation across all 15 stocks, providing a straightforward method to assess returns while maintaining diversification. It balances growth, stability, and sectoral representation, making it suitable for stress testing and long-term evaluation.

---

### Section 3.2

### Data Processing

#### Row-wise Exclusion for Portfolio Returns:
While calculating the portfolio return for each month, missing values (`NaN`) are ignored. This ensures that available data for other stocks contributes to the portfolio return without being biased by the absence of certain stocks in specific months. The `mean(axis=1, skipna=True)` method is used to average only non-missing values for each row (month).

#### Data Imputation for Sub-Portfolios or Prediction Models:
When grouping stocks into sub-portfolios or building predictive models, missing values in individual stock returns may need to be imputed to maintain consistency across observations. Possible approaches include:
- **Forward or Backward Filling**: Filling missing values with the most recent or subsequent available return for that stock.
- **Mean Imputation**: Replacing missing values with the average return of that stock across the dataset.
- **Industry or Sector-Based Imputation**: Replacing missing values with the average return of other stocks in the same industry or sector for that period.

#### Exclusion of Stocks with Excessive Missing Data:
Stocks with a high proportion of missing data (e.g., more than 50% of observations) may be excluded from sub-portfolio groupings or predictive models to avoid introducing excessive noise or bias.

## Strategy for Grouping and Prediction Models

### Grouping into Sub-Portfolios:
Grouping stocks into sub-portfolios based on meaningful classifications (e.g., industry, size, volatility) can help in creating more targeted predictive models.

#### Example groupings:
- **Industry-Based Clustering**: Group stocks by sectors like Consumer Staples, Financials, Healthcare, etc.
- **Market Cap-Based Clustering**: Divide stocks into large-cap, mid-cap, and small-cap categories.
- **Volatility-Based Clustering**: Group stocks with similar historical volatility or beta values.

### Prediction Models:

#### Sub-Portfolio Models:
**Pros**:
- Simplifies the problem by reducing the number of predictive models.
- Sub-portfolios can capture shared characteristics (e.g., sector trends or macroeconomic sensitivities).

**Cons**:
- Loss of granularity, as individual stock-specific patterns are averaged out.
- Requires careful selection of grouping criteria to avoid introducing bias.

#### Individual Stock Models:
**Pros**:
- Captures unique return dynamics of each stock.
- Allows for high precision in prediction if data is sufficient.

**Cons**:
- Computationally intensive for large portfolios.
- Performance may degrade if stocks have sparse or inconsistent data.

#### Single Portfolio-Wide Model:
**Pros**:
- Simplifies computation by treating the portfolio as a single entity.
- Reduces the need for imputation or clustering decisions.

**Cons**:
- Ignores individual stock or group-level differences.
- May perform poorly if the portfolio is highly heterogeneous.

---

### Section 3.3

### Data Sources

#### S&P 500 Monthly Returns: wrds
#### Portfolio Monthly Returns: wrds_data.xlsx

---

### Section 3.4

In [50]:
mev_data = pd.read_csv("2024-Table_2A_Historic_Domestic.csv")
mev_data

,Date,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,BBB corporate yield,Mortgage rate,Prime rate,Dow Jones Total Stock Market Index (Level),House Price Index (Level),Commercial Real Estate Price Index (Level),Market Volatility Index (Level)
0,1990 Q1,4.4,9.0,3.3,9.4,5.3,7.1,7.8,8.5,8.5,10.4,10.1,10.0,3273.5,76.0,108.4,27.3
1,1990 Q2,1.5,6.1,3.0,6.8,5.3,4.0,7.7,8.7,8.8,10.7,10.3,10.0,3424.4,76.0,107.5,24.2
2,1990 Q3,0.3,3.7,0.1,5.2,5.7,7.1,7.5,8.5,8.8,10.6,10.1,10.0,2879.3,75.9,107.0,36.5
3,1990 Q4,-3.6,-0.7,-3.2,2.1,6.1,7.0,7.0,8.1,8.5,10.9,10.0,10.0,3101.4,75.5,106.6,34.0
4,1991 Q1,-1.9,2.0,1.2,3.4,6.6,3.0,6.0,7.7,8.2,10.4,9.5,9.2,3583.7,74.9,105.6,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2022 Q4,2.6,6.5,2.2,6.4,3.6,4.2,4.0,4.1,3.9,6.1,6.7,6.8,38520.6,295.9,350.0,33.6
132,2023 Q1,2.2,6.3,10.8,15.5,3.5,3.8,4.6,3.8,3.7,5.6,6.4,7.7,41136.6,299.4,347.0,26.5
133,2023 Q2,2.1,3.8,3.3,5.8,3.6,2.7,5.1,3.7,3.7,5.7,6.5,8.2,44411.5,303.0,354.0,20.1
134,2023 Q3,4.9,8.3,0.3,2.9,3.7,3.6,5.3,4.3,4.2,6.0,7.0,8.4,42788.7,309.3,348.9,18.9


In [51]:
#mev_data['Date'] = pd.to_datetime(mev_data['Date'])
mev_data.set_index('Date', inplace=True)

def adf_test(series, significance=0.05):
    result = adfuller(series.dropna(), autolag='AIC')
    p_value = result[1]
    return p_value < significance  # True if stationary

transformations = {}

for col in mev_data.columns:
    print(f"Testing stationarity for: {col}")
    
    is_stationary = adf_test(mev_data[col])
    
    if is_stationary:
        print(f"{col}: Stationary (No transformation needed)")
        transformations[col] = mev_data[col]  
    else:
        diff_series = mev_data[col].diff()
        is_stationary_diff = adf_test(diff_series)
        
        if is_stationary_diff:
            print(f"{col}: Non-stationary, transformed using first difference")
            transformations[col] = diff_series
        else:
            if (mev_data[col] > 0).all():
                log_diff_series = np.log(mev_data[col]).diff()
                is_stationary_log_diff = adf_test(log_diff_series)
                
                if is_stationary_log_diff:
                    print(f"{col}: Non-stationary, transformed using log-first difference")
                    transformations[col] = log_diff_series
                else:
                    print(f"{col}: Non-stationary, transformation unsuccessful")
                    transformations[col] = log_diff_series
            else:
                print(f"{col}: Non-stationary, log transformation not possible due to non-positive values")
                transformations[col] = diff_series  # Use first difference as fallback

transformed_data = pd.DataFrame(transformations)
transformed_data.dropna(inplace=True)

transformed_data

Testing stationarity for: Real GDP growth
Real GDP growth: Stationary (No transformation needed)
Testing stationarity for: Nominal GDP growth
Nominal GDP growth: Stationary (No transformation needed)
Testing stationarity for: Real disposable income growth
Real disposable income growth: Stationary (No transformation needed)
Testing stationarity for: Nominal disposable income growth
Nominal disposable income growth: Stationary (No transformation needed)
Testing stationarity for: Unemployment rate
Unemployment rate: Stationary (No transformation needed)
Testing stationarity for: CPI inflation rate
CPI inflation rate: Stationary (No transformation needed)
Testing stationarity for: 3-month Treasury rate
3-month Treasury rate: Non-stationary, transformed using first difference
Testing stationarity for: 5-year Treasury yield
5-year Treasury yield: Non-stationary, transformed using first difference
Testing stationarity for: 10-year Treasury yield
10-year Treasury yield: Non-stationary, transfo

,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,BBB corporate yield,Mortgage rate,Prime rate,Dow Jones Total Stock Market Index (Level),House Price Index (Level),Commercial Real Estate Price Index (Level),Market Volatility Index (Level)
Date,,,,,,,,,,,,,,,,
1990 Q2,1.5,6.1,3.0,6.8,5.3,4.0,-0.1,0.2,0.3,0.3,0.2,10.0,150.9,0.000000,-0.9,24.2
1990 Q3,0.3,3.7,0.1,5.2,5.7,7.1,-0.2,-0.2,0.0,-0.1,-0.2,10.0,-545.1,-0.001317,-0.5,36.5
1990 Q4,-3.6,-0.7,-3.2,2.1,6.1,7.0,-0.5,-0.4,-0.3,0.3,-0.1,10.0,222.1,-0.005284,-0.4,34.0
1991 Q1,-1.9,2.0,1.2,3.4,6.6,3.0,-1.0,-0.4,-0.3,-0.5,-0.5,9.2,482.3,-0.007979,-1.0,36.2
1991 Q2,3.2,6.2,3.0,5.3,6.8,2.4,-0.4,0.1,0.1,-0.3,0.0,8.7,-38.2,0.007979,-1.0,20.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022 Q4,2.6,6.5,2.2,6.4,3.6,4.2,1.3,0.8,0.7,0.8,1.1,6.8,2422.6,0.000338,-1.0,33.6
2023 Q1,2.2,6.3,10.8,15.5,3.5,3.8,0.6,-0.3,-0.2,-0.5,-0.3,7.7,2616.0,0.011759,-3.0,26.5
2023 Q2,2.1,3.8,3.3,5.8,3.6,2.7,0.5,-0.1,0.0,0.1,0.1,8.2,3274.9,0.011952,7.0,20.1


The stationarity testing and transformations applied to the macroeconomic variables (MEVs) revealed that most growth rates (e.g., Real GDP growth, Nominal GDP growth, and Real disposable income growth) and rates (e.g., Unemployment rate, CPI inflation rate, and Prime rate) were inherently stationary and did not require transformation. For non-stationary variables like the 3-month Treasury rate, 5-year Treasury yield, 10-year Treasury yield, BBB corporate yield, Mortgage rate, and Dow Jones Total Stock Market Index, first differences were applied to remove trends and achieve stationarity. These variables often exhibit linear trends, which are effectively addressed by differencing.

The House Price Index exhibited exponential growth, making log-first differences a suitable transformation. This approach captures proportional changes and removes non-stationary trends in the level data. Similarly, the Commercial Real Estate Price Index, while non-stationary, was transformed using first differences due to its linear trend. The Market Volatility Index, despite being a level variable, was stationary and required no transformation.

In summary, the choice of transformation depended on the nature of the variable: first differences were used for variables with linear trends, while log-first differences were applied for variables with exponential growth. These transformations ensure the data is stationary, a necessary condition for robust time-series analysis and predictive modeling.

---

### Section 3.5

In [54]:
summary_statistics = transformed_data.describe()
summary_statistics.T

,count,mean,std,min,25%,50%,75%,max
Real GDP growth,135.0,2.528148,4.546700,-28.000000,1.450000,2.500000,4.000000,34.800000
Nominal GDP growth,135.0,4.842963,5.104922,-29.200000,3.600000,5.000000,6.500000,39.700000
Real disposable income growth,135.0,2.807407,7.566754,-27.600000,1.200000,2.700000,4.250000,56.000000
Nominal disposable income growth,135.0,5.028148,7.574985,-23.100000,3.350000,5.000000,6.400000,63.500000
Unemployment rate,135.0,5.761481,1.748986,3.500000,4.450000,5.400000,6.700000,13.000000
CPI inflation rate,135.0,2.660000,2.244768,-8.900000,1.800000,2.800000,3.550000,9.700000
3-month Treasury rate,135.0,-0.018519,0.427800,-1.300000,-0.100000,0.000000,0.200000,1.600000
5-year Treasury yield,135.0,-0.029630,0.415030,-1.100000,-0.300000,0.000000,0.200000,1.200000
10-year Treasury yield,135.0,-0.029630,0.366727,-0.900000,-0.300000,0.000000,0.200000,1.000000
BBB corporate yield,135.0,-0.031111,0.483375,-1.600000,-0.300000,0.000000,0.200000,2.600000


The summary statistics indicate the characteristics of the macroeconomic variables (MEVs) used in the analysis. Most variables have reasonable ranges, reflecting both economic expansions and contractions over the covered time period. For instance, Real GDP growth and Nominal GDP growth show mean values of approximately 2.53% and 4.84%, respectively, which align with typical economic growth rates over long periods. However, these variables also show significant variability, with minimum values of -28% and -29.2%, reflecting severe economic downturns such as recessions or crises. The maximum growth rates, particularly for Nominal GDP at 39.7%, may reflect periods of rapid expansion or high inflation.

The Unemployment rate exhibits relatively stable variability, with a mean of 5.76% and a standard deviation of 1.75%. Its range (3.5% to 13%) captures historical lows during economic booms and peaks during periods of unemployment surges, such as recessions. Similarly, the CPI inflation rate has a mean of 2.66% with manageable variability, consistent with historical inflation targets. However, its minimum of -8.9% suggests deflationary periods, while the maximum of 9.7% aligns with inflationary spikes, such as during the 1970s and 1980s.

In financial indicators, Treasury yields (3-month, 5-year, and 10-year) and corporate yields exhibit close-to-zero mean values, reflecting long periods of low interest rates. Their ranges highlight periods of high-rate environments and near-zero interest rate policies. The Dow Jones Total Stock Market Index, with its large standard deviation and range (min: -7870.8, max: 5592.0), captures the volatility of equity markets over time, including significant downturns such as the 2008 financial crisis and recoveries thereafter. The House Price Index and Commercial Real Estate Price Index show relatively low means and standard deviations, indicative of steady growth punctuated by disruptions like the housing crisis.

Overall, these statistics highlight the diversity and volatility inherent in macroeconomic and financial indicators. The wide variability in certain metrics underscores the importance of robust statistical modeling to capture the dynamics and mitigate the impact of outliers in stress-testing scenarios. Each variable provides critical insight into different dimensions of economic performance, which is essential for a comprehensive portfolio risk analysis.

Time Period being used for analysis: 1990 Q2 to 2023 Q4.

---